## web crawling

In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver_service = Service(ChromeDriverManager().install())

url = "https://www.google.com/search?sca_esv=a3944c8b8683a63b&hl=ko&sxsrf=ADLYWILYk3pz3PF-muVX-JQ2MTe1DdsohA:1736956033989&q=%EA%B3%A0%EC%86%8D%EB%8F%84%EB%A1%9C+%EC%9E%90%EB%8F%99%EC%B0%A8+%EC%82%BC%EA%B0%81%EB%8C%80&uds=ADvngMjcH0KdF7qGWtwTBrP0nt7dNJkbpgjqZNv4hs5fyca8-GSNikXodhMYrK9jlr_SlYmmsjrDLK6Gqhz6k_NBrD44AJ0nvRS5x1wpUeXRc9cAv0PvCkZvT5TxnuJ8_EpPN9Ja_hGC01M2gHhcXx9SHePMd-KCe1sZBp5oTL2f7kR4e3rXWdUb61YvuBtdCD_KXP1KPgDU&udm=2&sa=X&ved=2ahUKEwjwyZW7ifiKAxXtsFYBHZexAVYQxKsJegQICBAB&ictx=0&biw=1440&bih=731&dpr=2"

driver = webdriver.Chrome(service=driver_service, options=options)
driver.get(url)

SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.quit()

images = soup.find_all('img')
image_urls = [img['src'] for img in images if 'src' in img.attrs]

save_dir = "downloaded_images"
os.makedirs(save_dir, exist_ok=True)

def download_image(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
    except Exception as e:
        print(f"Error downloading {url}: {e}")

for i, img_url in enumerate(image_urls):
    try:
        if not img_url.startswith('http'):
            continue
        filename = os.path.join(save_dir, f"image_{i+1}.jpg")
        download_image(img_url, filename)
        print(f"Downloaded: {filename}")
    except Exception as e:
        print(f"Failed to download image {i+1}: {e}")

print("All images downloaded.")
